In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import time
from scipy.stats import skew, boxcox
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from bayes_opt import BayesianOptimization

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def xg_eval_mae(yhat, dtrain, lift=0):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-lift, np.exp(yhat)-lift)

def xgb_logregobj(preds, dtrain):
    con = 2
    labels = dtrain.get_label()
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess

# Load data

In [3]:
start = time.time() 
train_data = pd.read_csv('../input/train.csv')
train_size=train_data.shape[0]
print ("Loading train data finished in %0.3fs" % (time.time() - start))

start = time.time()
test_data = pd.read_csv('../input/test.csv')
print ("Loading test data finished in %0.3fs" % (time.time() - start))   

Loading train data finished in 3.027s
Loading test data finished in 1.927s


#### Merge train and test
Save our time on duplicating logics for train and test and will also ensure the transformations applied on train and test are the same.

In [4]:
full_data=pd.concat([train_data,test_data])
del( train_data, test_data)
print ("Full Data set created.")

Full Data set created.


In [5]:
data_types = full_data.dtypes  
cat_cols = list(data_types[data_types=='object'].index)
num_cols = list(data_types[data_types=='int64'].index) + list(data_types[data_types=='float64'].index)

id_col = 'id'
target_col = 'loss'
num_cols.remove('id')
num_cols.remove('loss')

## Numeric features

Two preprocessings on numeric features are applied:

1. Apply box-cox transformations for skewed numeric features.

2. Scale numeric features so they will fall in the range between 0 and 1.

In [7]:
skewed_cols = full_data[num_cols].apply(lambda x: skew(x.dropna()))

SSL = preprocessing.StandardScaler()
skewed_cols = skewed_cols[skewed_cols > 0.25].index.values
for skewed_col in skewed_cols:
    full_data[skewed_col], lam = boxcox(full_data[skewed_col] + 1)
for num_col in num_cols:
    full_data[num_col] = SSL.fit_transform(full_data[num_col].values.reshape(-1,1))

# Model LE Coding
### Categorical features
1. Label Encoding (Factorizing)

In [6]:
LBL = preprocessing.LabelEncoder()
start=time.time()
for cat_col in cat_cols:
    full_data[cat_col] = LBL.fit_transform(full_data[cat_col])
print ('Label enconding finished in %f seconds' % (time.time()-start))

Label enconding finished in 37.017625 seconds


In [10]:
lift = 200

train_x = full_data[cat_cols + num_cols][:train_size]
test_x = full_data[cat_cols + num_cols][train_size:]
train_y = np.log(full_data[:train_size].loss.values + lift)
ID = full_data.id[:train_size].values

xgtrain = xgb.DMatrix(train_x, label=train_y) #used for Bayersian Optimization


X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, train_size=.80, random_state=1234)

## [XGBoost](http://xgboost.readthedocs.io/en/latest/) tuning


### Manual tuning 
1. [Bayersian Optimization](https://github.com/fmfn/BayesianOptimization) will be introduced to optimize each parameter. 
2. Based on holdout data (X_val, y_val) for validation for the sake of time. The range for each parameter can be estimated based on the manual tuning results. However, grid search can also be used if time allows.
3. A larger learning rate (0.1) is implemented since it requires less iterations to complete. A smaller one (0.01 or even smaller) will be used to train the model for better accuracy when the parameters are optimized.

#### 1. Tune max_depth
**max_depth : int**

    Maximum tree depth for base learners.

In [18]:
%%time
learning_rate = 0.1
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth= 5,
    nthread = -1,
    silent = False
)
rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=20,
    verbose=False
)

print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
print 'score    :\t{}\n'.format(rgr.best_score)

max_depth:	5

score:		1154.561768

Wall time: 2min 56s


#### The results are listed as follow:

    max_depth  score
    5          1154.56
    6          1156.13
    7          1155.86
    8          1156.74
    9          1158.39
    10         1161.08
    11         1164.82
    12         1168.76
    4          1157.56 
    3          1163.53

In [19]:
max_depth = 5

#### 2. Tune min_child_weight
**min_child_weight : int**
    
    Minimum sum of instance weight(hessian) needed in a child.

In [20]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=105,
    nthread = -1,
    silent = False
)
rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)
print 'min_child_weight:\t{}\n'.format(rgr.get_xgb_params()['min_child_weight'])
print 'score           :\t{}\n'.format(rgr.best_score)

min_child_weight:	105

score           :	1150.430908

Wall time: 4min 48s


#### The results are listed as follow:

    min_child_weight  score
    1                 1153.27
    5                 1155.26
    10                1156.5
    15                1155.37
    20                1153.96
    25                1155.06
    30                1152.18
    35                1153.08
    40                1154.31
    45                1153.62
    50                1153.89
    55                1153.72
    60                1152.41
    65                1152.57
    70                1152.41
    75                1152.51
    80                1152.07
    85                1152.13
    90                1152.45
    95                1151.88
    100               1153.39
    105               1150.43
    110               1152.67
    115               1151.76
    120               1150.9
    125               1152.07
    130               1151.77

In [21]:
min_child_weight = 105

#### 3.Tune colsample_bytree

**colsample_bytree : float**

    Subsample ratio of columns when constructing each tree.

In [22]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=0.8,
    nthread = -1,
    silent = False
)


rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)
print 'colsample_bytree:\t{}\n'.format(rgr.get_xgb_params()['colsample_bytree'])
print 'score           :\t{}\n'.format(rgr.best_score)

colsample_bytree:	0.8

score           :	1153.147461

Wall time: 4min 8s


#### The results are listed as follow:

    colsample_bytree  score
    1                 1150.43
    0.9               1150.85
    0.8               1153.15
    0.7               1154.88
    0.6               1153.1
    0.5               1153.89
    0.4               1152.61
    0.3               1152.56

In [24]:
# The value 0.8 was chosen to introduce randomization 
# since it is the objective to find the range of each parameter.
colsample_bytree = 0.8 

#### 4.Tune subsample
**subsample : float**

    Subsample ratio of the training instance.

In [26]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree,
    subsample=0.8,
    nthread = -1,
    silent = False
)


rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)

print 'subsample:\t{}\n'.format(rgr.get_xgb_params()['subsample'])
print 'score    :\t{}\n'.format(rgr.best_score)

subsample:	0.8

score    :	1153.380859

Wall time: 2min 51s


    subsample score
    1         1153.15
    0.9       1150.88
    0.8       1153.38
    0.7       1154.21
    0.6       1156.15

In [27]:
# The value 0.8 was chosen to introduce randomization 
# since it is the objective to find the range of each parameter.
subsample = 0.8

#### 5. Tune gamma
**gamma : float**

    Minimum loss reduction required to make a further partition on a leaf node of the tree.

In [28]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234,
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree,
    subsample=subsample,
    gamma=1.2,
    nthread = -1,
    silent = False
)

rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)


print 'gamma:\t{}\n'.format(rgr.get_xgb_params()['gamma'])
print 'score:\t{}\n'.format(rgr.best_score)

gamma:	1.2

score:	1151.082642

Wall time: 4min 55s


    gamma score
    0     1153.38
    0.3   1153.13
    0.6   1151.36
    0.9   1152.17
    1.2   1151.08
    1.5   1151.47
    1.8   1154.93
    2.1   1152.41
    2.4   1154.99
    2.7   1156.14
    3     1156.72

In [29]:
gamma = 1.2

### Automated tuning - [Bayesian Optimization](https://github.com/fmfn/BayesianOptimization)

The idea is to set a range for each parameters, for which we can leverage the parameters from manual tuning, then let the bayersian optimization to seek best parameters.

It's more efficient than grid search but is still time consuming, which took me over 20 hours to finish the optimization. Therefore knowing an approximate range of values for each parameter will greatly improve the performance.

In [ ]:
def xgb_evaluate(min_child_weight, colsample_bytree, max_depth, subsample, gamma):
    params = dict()
    params['eta'] = 0.1
    params['verbose_eval'] = True
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    
    cv_result = xgb.cv(
        params, xgtrain, 
        num_boost_round=10000, nfold=4,
        feval = xg_eval_mae,obj = xgb_logregobj,
        seed=1234,callbacks=[xgb.callback.early_stop(50)]
    )
    
    return -cv_result['test-mae-mean'].values[-1]


xgb_BO = BayesianOptimization(
    xgb_evaluate, 
    {
        'max_depth': (max_depth - 1, max_depth + 3),
        'min_child_weight': (min_child_weight - 20, min_child_weight + 20),
        'colsample_bytree': (max(colsample_bytree - 0.2, 0.1), min(colsample_bytree + 0.2, 1)),
        'subsample': (max(subsample - 0.2, 0.1), min(subsample + 0.2, 1)),
        'gamma': (max(gamma - 0.25, 0), gamma + 0.4)
    }
)

xgb_BO.maximize(init_points=5, n_iter=25)

### The best sets of parameters are listed:

# Model OHE Coding
####  Categorical features
1. Label Encoding (Factorizing)
2. One Hot Encoding (get dummies)

OHE can be done by either get_dummies() from Pandas package or OneHotEncoder from SK-Learn package. 

* get_dummies is easier to implement (can be used directly on raw categorical features, i.e. strings, but it takes longer time and is not memory efficient.

* OneHotEncoder requires the features being converted to numeric, which has already been done by LabelEncoder in previous step, and is much more efficient (7x faster).

* The OHE's results are converted to a sparse matrix which uses way less memory as compared to dense matrix. 

In [ ]:
# This step has been finished previously. Run this cell if start with fresh data.


# start = time.time() 
# train_data = pd.read_csv('../input/train.csv')
# train_size=train_data.shape[0]
# print ("Loading train data finished in %0.3fs" % (time.time() - start))

# start = time.time()
# test_data = pd.read_csv('../input/test.csv')
# print ("Loading test data finished in %0.3fs" % (time.time() - start))   

# full_data=pd.concat([train_data,test_data])
# del( train_data, test_data)
# print ("Full Data set created.")

# data_types = full_data.dtypes  
# cat_cols = list(data_types[data_types=='object'].index)
# num_cols = list(data_types[data_types=='int64'].index) + list(data_types[data_types=='float64'].index)

# id_col = 'id'
# target_col = 'loss'
# num_cols.remove('id')
# num_cols.remove('loss')

# LBL = preprocessing.LabelEncoder()
# start=time.time()
# for cat_col in cat_cols:
# #     print ("Factorize feature %s" % (cat))
#     full_data[cat_col] = LBL.fit_transform(full_data[cat_col])
# print ('Label enconding finished in %f seconds' % (time.time()-start))



In [7]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
full_data_sparse=OHE.fit_transform(full_data[cat_cols])
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))
print (full_data_sparse.shape)

## it should be (313864, 1176)

One-hot-encoding finished in 10.411293 seconds
(313864, 1176)


In [9]:
from scipy import sparse

lift = 200

full_data_sparse = sparse.hstack((full_data_sparse
                                  ,full_data[num_cols])
                                 , format='csr'
                                 )
print (full_data_sparse.shape)
train_x = full_data_sparse[:train_size]
test_x = full_data_sparse[train_size:]
train_y = np.log(full_data[:train_size].loss.values + lift)
ID = full_data.id[:train_size].values

xgtrain = xgb.DMatrix(train_x, label=train_y) #used for Bayersian Optimization

from sklearn.cross_validation import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, train_size=.80, random_state=1234)

(313864, 1190)


## [XGBoost](http://xgboost.readthedocs.io/en/latest/) tuning

#### 1. Tune max_depth
**max_depth : int**

    Maximum tree depth for base learners.

In [ ]:
learning_rate = 0.1
rgr = xgb.XGBRegressor(
    seed = 1234, 
    objective = logregobj,
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=13,
    nthread = -1,
    silent = False
)
rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=20,
    verbose=False
)

print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
print 'score    :\t{}\n'.format(rgr.best_score)

    max_depth score
    5         1163.51
    6         1167.68
    7         1169.18 
    8         1169.68 
    10        1175.96 
    15        1189.57 
    12        1177.27
    11        1175.76
    9         1172.38
    13        1180.775757

In [10]:
max_depth = 5

#### 2. Tune min_child_weight
**min_child_weight : int**
    
    Minimum sum of instance weight(hessian) needed in a child.

In [ ]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=105,
    nthread = -1,
    silent = False
)
rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)
print 'min_child_weight:\t{}\n'.format(rgr.get_xgb_params()['min_child_weight'])
print 'score           :\t{}\n'.format(rgr.best_score)

    min_child_weight score 
    15               1157.88
    5                1161.54
    1                1163.46
    10               1161.81
    20               1159.91
    25               1158.85
    30               1158.3
    40               1155.78
    45               1158.47
    50               1157.56
    55               1157.01
    60               1156.45
    65               1158.1

In [ ]:
min_child_weight = 40

#### 3.Tune colsample_bytree

**colsample_bytree : float**

    Subsample ratio of columns when constructing each tree.

In [ ]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=0.8,
    nthread = -1,
    silent = False
)


rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)
print 'colsample_bytree:\t{}\n'.format(rgr.get_xgb_params()['colsample_bytree'])
print 'score           :\t{}\n'.format(rgr.best_score)

    colsample_bytree score
    1                1155.78
    0.9              1164.98
    0.8              1160.84
    0.7              1164.93
    0.6              1157.09
    0.5              1159.3
    0.4              1158.58
    0.3              1160.91
    0.2              1155.38
    0.1              1159.3

In [ ]:
colsample_bytree = 0.2 #

#### 4.Tune subsample
**subsample : float**

    Subsample ratio of the training instance.

In [ ]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234, # use a fixed seed during tuning so we can reproduce the results
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree,
    subsample=0.8,
    nthread = -1,
    silent = False
)


rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)

print 'subsample:\t{}\n'.format(rgr.get_xgb_params()['subsample'])
print 'score    :\t{}\n'.format(rgr.best_score)

    subsample score
    1         1155.38
    0.9       1157.38
    0.8       1154.79
    0.7       1157.225586
    0.6       1158.565796

In [ ]:
subsample =  0.8 

#### 5. Tune gamma
**gamma : float**

    Minimum loss reduction required to make a further partition on a leaf node of the tree.

In [ ]:
%%time
rgr = xgb.XGBRegressor(
    seed = 1234,
    learning_rate = learning_rate,
    n_estimators = 10000,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree,
    subsample=subsample,
    gamma=1.2,
    nthread = -1,
    silent = False
)

rgr.fit(
    X_train,y_train,
    eval_set=[(X_val,y_val)],
    eval_metric=xg_eval_mae,
    early_stopping_rounds=50,
    verbose=False
)


print 'gamma:\t{}\n'.format(rgr.get_xgb_params()['gamma'])
print 'score:\t{}\n'.format(rgr.best_score)

    gamma score
    0     1154.79
    0.3   1154.103394
    0.6   1153.088135
    0.9   1155.900391
    1.2   1156.186035
    1.5   1155.787231
    1.8   1158.119263
    2.1   1160.725952
    2.4   1164.15

In [ ]:
gamma = 0.6 

### Automated tuning - [Bayesian Optimization](https://github.com/fmfn/BayesianOptimization)

The idea is to set a range for each parameters, for which we can leverage the parameters from manual tuning, then let the bayersian optimization to seek best parameters.

It's more efficient than grid search but is still time consuming, which took me over 20 hours to finish the optimization. Therefore knowing an approximate range of values for each parameter will greatly improve the performance.

In [ ]:
def xgb_evaluate(min_child_weight,colsample_bytree,max_depth,subsample,gamma):
    params = {
    'eta': 0.1,
    'silent': 1,
    'verbose_eval': True,
    'seed': 1234
    }
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    cv_result = xgb.cv(
        params, xgtrain, num_boost_round=10000, nfold=4,
        obj = xgb_logregobj, feval=xg_eval_mae,
        seed=1234,callbacks=[xgb.callback.early_stop(50)])
    return -cv_result['test-mae-mean'].values[-1]





xgb_BO = BayesianOptimization(
    xgb_evaluate, 
    {'max_depth': (11 , 18),
     'min_child_weight': (100, 200),
     'colsample_bytree': (0.05, 0.3),
     'subsample': (0.6, 1),
     'gamma': (0.5, 2.0)
    }
)

xgb_BO.initialize({
        -1147.727325: {'max_depth': 5.000000, 'min_child_weight': 60.000000, 'subsample':0.800000,
                      'colsample_bytree':0.800000, 'gamma':0.000000},
        -1143.200348: {'max_depth': 11.312448, 'min_child_weight': 119.388715, 'subsample':0.916137,
                      'colsample_bytree':0.209870, 'gamma':0.564498},
        -1141.388184: {'max_depth': 14.000000, 'min_child_weight': 140.000000, 'subsample':1.000000,
                      'colsample_bytree':0.050000, 'gamma':0.900000},
        
    })

xgb_BO.maximize(init_points=5, n_iter=25)

### The best sets of parameters are listed: